In [2]:
import os
import numpy as np
import audiolabel
import pandas as pd
import parselmouth as pm

In [3]:
testwav = './SEG_07_13_2011_JEO_insects-pronouns-colors_part.wav'
wav = pm.Sound(testwav)
[au] = wav.extract_all_channels()  # splitting the channels (Step 1!)
au

In [10]:
testtg = './SEG_07_13_2011_JEO_insects-pronouns-colors_part.TextGrid'

In [11]:
tg = audiolabel.LabelManager(from_file=testtg, from_type='praat')
tg

LabelManager(tiers=[0,1,2,3,4,5], names=['segment','phonation','word','translation','phrase','dictionary'])

In [13]:
vowels = ['ɪ', 'a', 'i', 'o', 'e']

In [14]:
tgdf = tg.as_df()[0]
voweldf = tgdf[tgdf.text.isin(vowels)]
voweldf.head()

,t1,t2,text,duration,center
1,0.806516,0.937395,ɪ,0.130879,0.871955
3,1.070030,1.302008,a,0.231978,1.186019
7,1.832930,1.960632,ɪ,0.127702,1.896781
9,2.070831,2.219467,a,0.148637,2.145149
12,4.739548,4.930282,ɪ,0.190734,4.834915


In [42]:
tg.tier('phonation').label_at(12.2).text

'M!'

In [26]:
voweldf

,t1,t2,text,duration,center
1,0.806516,0.937395,ɪ,0.130879,0.871955
3,1.070030,1.302008,a,0.231978,1.186019
7,1.832930,1.960632,ɪ,0.127702,1.896781
9,2.070831,2.219467,a,0.148637,2.145149
12,4.739548,4.930282,ɪ,0.190734,4.834915
14,5.100564,5.357290,a,0.256726,5.228927
18,12.118474,12.277926,ɪ,0.159452,12.198200
22,13.295197,13.468001,e,0.172804,13.381599


In [19]:
timepoints = [.1, .3, .5, .7, .9]

In [43]:
f1ss = []
f2ss = []
f3ss = []
phonss = []

for index, row in voweldf.iterrows():
    start = row.t1
    end = row.t2
    
    formants = au.to_formant_burg(max_number_of_formants=5.0, maximum_formant=6000)  # analog of the To Formant command (Step 2!)

    f1s = f2s = f3s = phons = []
    for t in timepoints:
        time = row.duration*t+start # getting actual time
        f1 = formants.get_value_at_time(formant_number=1, time=time)
        f2 = formants.get_value_at_time(formant_number=2, time=time)
        f3 = formants.get_value_at_time(formant_number=3, time=time)
        phon = tg.tier('phonation').label_at(time).text
        
        f1s = f1s + [f1]
        f2s = f2s + [f2]
        f3s = f3s + [f3]
        phons = phons + [phon]
        
    f1ss = f1ss + [f1s]
    f2ss = f2ss + [f2s]
    f3ss = f3ss + [f3s]
    phonss = phonss + [phons]
        
voweldf['f1s'] = f1ss
voweldf['f2s'] = f2ss
voweldf['f3s'] = f3ss
voweldf['phons'] = phonss

voweldf

<ipython-input-43-a0e362eda819>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  voweldf['f1s'] = f1ss
<ipython-input-43-a0e362eda819>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  voweldf['f2s'] = f2ss
<ipython-input-43-a0e362eda819>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

,t1,t2,text,duration,center,f1s,f2s,f3s,phons
1,0.806516,0.937395,ɪ,0.130879,0.871955,"[474.5114834883744, 488.866649153624, 496.0778...","[2349.0287037187836, 2445.087664117453, 2579.9...","[3079.4961838042154, 3212.8824002434, 3169.118...","[M!, M!, M!, M!, M!]"
3,1.070030,1.302008,a,0.231978,1.186019,"[804.1921947246527, 803.3458150533991, 857.055...","[2184.7210459353423, 1782.275723850941, 1731.8...","[2759.5771221174646, 2970.3418449885685, 3128....","[M!, M!, M!, M!, M!]"
7,1.832930,1.960632,ɪ,0.127702,1.896781,"[411.5170982733097, 469.9749704059344, 482.022...","[2678.8389446166193, 2862.095435417448, 2552.2...","[3075.4564337709985, 3049.323268835495, 3317.5...","[C, C, M, M, M]"
9,2.070831,2.219467,a,0.148637,2.145149,"[752.1881865465117, 787.6154491613393, 806.547...","[2396.26019981286, 1962.4667234277308, 1860.27...","[3039.0986674792885, 2882.7289477287713, 2944....","[M!, M!, M!, M!, M!]"
12,4.739548,4.930282,ɪ,0.190734,4.834915,"[374.14029938196893, 362.4508747883075, 459.79...","[2278.775211943429, 2568.856087565332, 2559.50...","[3078.2144424290677, 3240.430872101625, 3157.4...","[M!, M!, M!, M!, M!]"
14,5.100564,5.357290,a,0.256726,5.228927,"[781.9985913486522, 802.978099159896, 786.2033...","[2146.5001920815503, 1889.5197690685693, 1899....","[3016.8900897372127, 2995.4832397776645, 3009....","[M!, M!, M!, M!, M!]"
18,12.118474,12.277926,ɪ,0.159452,12.198200,"[436.4039501108636, 538.9517728840494, 538.996...","[1381.498945005181, 2069.305072619901, 2385.90...","[2675.753285314365, 2825.265586701906, 3077.15...","[M!, M!, M!, M!, M!]"
22,13.295197,13.468001,e,0.172804,13.381599,"[489.69913931219054, 478.3703518429815, 378.31...","[1748.0878706424285, 2037.4198590427766, 2210....","[2898.3179310225646, 3177.3663164715535, 3092....","[M!, M!, M!, M!, M!]"
